# IMPORT DATA

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

df = pd.read_csv('result/data/melting_point_features.csv')

y = df['Tm']
X = df.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=2601)

base_model = LGBMRegressor(random_state=2601, n_jobs=1)

# REMOVE OUTLIERS

In [2]:
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

train_data = X_train.copy()
train_data['Tm'] = y_train

print(f"Original Train Size: {len(train_data)}")

train_data_clean = remove_outliers(train_data, 'Tm')

print(f"Cleaned Train Size: {len(train_data_clean)}")
print(f"Removed: {len(train_data) - len(train_data_clean)} Samp")

X_train_clean = train_data_clean.drop(columns=['Tm'])
y_train_clean = train_data_clean['Tm']

Original Train Size: 8416
Cleaned Train Size: 8115
Removed: 301 Samp


# RFECV Train

In [ ]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
import time
import warnings
warnings.filterwarnings('ignore')

print("\n---START RFE ---")
start = time.time()

model = LGBMRegressor(
    objective='regression_l1',
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=2601,
    n_jobs=1,
    verbose=-1)

rfe = RFECV(
    estimator=model, 
    min_features_to_select=50,
    step=20,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,
    verbose=1
)

rfe.fit(X_train, y_train)

selected_rfe = X_train.columns[rfe.support_]
print(f"⏱️ Time Run: {time.time() - start:.2f} s")
print(f"✅ RFECV Choosen {len(selected_rfe)} features:")
print(list(selected_rfe))

import joblib
joblib.dump(list(selected_rfe), 'result/rfe_features.pkl')

# GENETIC ALGORITHM (GA) Train

In [3]:
import warnings
import time
from sklearn_genetic import GAFeatureSelectionCV
from lightgbm import LGBMRegressor

warnings.filterwarnings('ignore')

print("\n--- 🧬 START RUN GENETIC ALGORITHM ---")
start = time.time()
model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    max_depth=-1,
    random_state=2601,
    n_jobs=1, verbose=-1
    )

ga = GAFeatureSelectionCV(
    estimator=model,
    cv=3,
    scoring="neg_root_mean_squared_error",
    population_size=50,    
    generations=15,
    mutation_probability=0.1,
    crossover_probability=0.8,
    keep_top_k=2,
    elitism=True,
    n_jobs=-1,
    verbose=True
)

ga.fit(X_train, y_train)

selected_ga = X_train.columns[ga.support_]

print(f"⏱️ Time Run: {time.time() - start:.2f} s")
print(f"\n✅ GA Choosen {len(selected_ga)} features:")
print("\List Features:")
print(list(selected_ga))

import joblib
joblib.dump(list(selected_ga), 'result/ga_features.pkl')


--- 🧬 START RUN GENETIC ALGORITHM ---
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	-267.764	4.93195    	-255.101   	-278.156   
1  	92    	-263.454	3.98779    	-255.101   	-270.855   
2  	93    	-259.998	2.17209    	-255.489   	-265.386   
3  	90    	-258.399	1.52927    	-255.489   	-261.992   
4  	94    	-257.449	1.35679    	-254.948   	-260.049   
5  	89    	-256.46 	1.14056    	-253.791   	-258.827   
6  	85    	-256.161	0.848217   	-253.791   	-257.802   
7  	83    	-255.742	1.04772    	-252.321   	-258.328   
8  	88    	-255.79 	0.915433   	-253.791   	-258.328   
9  	86    	-255.424	0.788329   	-254.108   	-257.596   
10 	93    	-255.258	1.12705    	-252.941   	-257.705   
11 	91    	-254.679	1.07367    	-252.941   	-257.705   
12 	89    	-254.393	0.989895   	-252.941   	-256.391   
13 	91    	-254.473	1.02966    	-252.941   	-256.832   
14 	94    	-254.008	0.748887   	-251.927   	-255.395   
15 	93    	-253.849	0.792186   	-252.798   	-256.287   
⏱️ Time R

['result/ga_features.pkl']

# UNION 2 MODEL

In [7]:
import joblib

rfe_features = joblib.load('result/rfe_features.pkl')
ga_features = joblib.load('result/ga_features.pkl')

common_features = set(rfe_features) | set(ga_features)

print(f"\n💎 Tổng features sau khi gộp (Union): {len(common_features)}")
print(list(common_features))


💎 Tổng features sau khi gộp (Union): 632
['NumHDonors', 'Count_C', 'MACCS_94', 'fr_para_hydroxylation', 'Morgan_117', 'Morgan_157', 'Morgan_260', 'FracSingle', 'Morgan_440', 'MACCS_67', 'MaxPartialCharge', 'Morgan_397', 'MACCS_3', 'Morgan_189', 'HeavyAtomMolWt', 'fr_dihydropyridine', 'MACCS_23', 'MACCS_144', 'Morgan_250', 'Morgan_482', 'SMI_len', 'Morgan_233', 'Morgan_217', 'Morgan_415', 'fr_allylic_oxid', 'fr_NH1', 'Rings56_frac', 'Morgan_376', 'Morgan_416', 'Morgan_323', 'Morgan_509', 'Morgan_321', 'Morgan_443', 'SMI_ringDigits', 'MinPartialCharge', 'Morgan_421', 'NHOHCount', 'Morgan_134', 'Morgan_64', 'PEOE_VSA13', 'Morgan_267', 'Gasteiger_q_abs_sum', 'Morgan_377', 'MACCS_121', 'NumRadicalElectrons', 'MACCS_164', 'MACCS_106', 'Chi2v', 'MACCS_59', 'PEOE_VSA3', 'MACCS_15', 'SlogP_VSA1', 'RingDensity_perHeavyAtom', 'Morgan_124', 'MACCS_53', 'MACCS_150', 'Morgan_192', 'SlogP_VSA0', 'Morgan_68', 'Morgan_135', 'Morgan_493', 'Morgan_438', 'Morgan_394', 'RingCount', 'Morgan_1', 'Morgan_427

# CHOICE BEST FEATURES

In [8]:
import joblib
from lightgbm import LGBMRegressor

best_features = list(rfe_features)

print(f"✅ Đang train model cuối cùng với {len(best_features)} features...")

manual_params = {
    'n_estimators': 3000,
    'learning_rate': 0.01,
    'num_leaves': 60,
    'max_depth': 15,
    'objective': 'regression_l1',
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'random_state': 2601,
    'n_jobs': 1,
    'verbose': -1
}

final_model = LGBMRegressor(**manual_params)

final_model.fit(X_clean[best_features], y)

# Lưu model và danh sách features
joblib.dump(final_model, 'result/final_melting_point_model.pkl')
joblib.dump(best_features, 'result/final_features_list.pkl')

print("💾 Đã lưu model và features thành công!")

✅ Đang train model cuối cùng với 297 features...
💾 Đã lưu model và features thành công!


# SCORING

## Score Combine Features

In [9]:
import pandas as pd
import numpy as np
import joblib
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import gc
model = joblib.load('result/final_melting_point_model.pkl')
features = joblib.load('result/final_features_list.pkl')

df = pd.read_csv('result/data/melting_point_features.csv')

needed_cols = list(features) + ['Tm']

existing_cols = [c for c in needed_cols if c in df.columns]

df_reduced = df[existing_cols].copy()

del df
gc.collect()

y = df_reduced['Tm']
X = df_reduced.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.mask(X > 1e308, np.nan)

print("⚙️(Imputing)...")
imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

_, X_test, _, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=2601)

y_pred = model.predict(X_test[features])

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- 🏁 RESULT ---")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

⚙️(Imputing)...

--- 🏁 RESULT ---
MAE: 76.1625
RMSE: 279.5836
R2: 0.5021


## Score each Model

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')
y = df['Tm']
X = df.drop(columns=['Tm']).select_dtypes(include=[np.number])

X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.mask(X > 1e308, np.nan)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2)

def get_metrics(name, feature_list):
    valid_feats = [f for f in feature_list if f in X_train.columns]
    
    if not valid_feats: return {"Method": name, "Features": 0, "RMSE": 0, "R2": 0}

    model = LGBMRegressor(      
        n_jobs=1,
        verbose=-1,
        n_estimators=2000,
        learning_rate=0.01,
        num_leaves=50,
        max_depth=-1)
    model.fit(X_train[valid_feats], y_train)
    y_pred = model.predict(X_test[valid_feats])
    
    return {
        "Method": name,
        "Features": len(valid_feats),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2": r2_score(y_test, y_pred)
    }

feats_all = list(X_train.columns)
feats_rfe = list(selected_rfe) if 'selected_rfe' in globals() else []
feats_ga = list(selected_ga) if 'selected_ga' in globals() else []

results = []
results.append(get_metrics("Original", feats_all))
results.append(get_metrics("RFE", feats_rfe))
results.append(get_metrics("GA", feats_ga))

df_res = pd.DataFrame(results)
base_rmse = df_res.loc[0, 'RMSE']
base_r2 = df_res.loc[0, 'R2']

df_res['Diff_RMSE'] = df_res['RMSE'] - base_rmse
df_res['Diff_R2'] = df_res['R2'] - base_r2

print(df_res.round(4))

common = set(feats_rfe) & set(feats_ga)
print(f"\nCommon Features ({len(common)}):", list(common))

     Method  Features      RMSE      R2  Diff_RMSE  Diff_R2
0  Original       937  249.9172  0.6190     0.0000   0.0000
1       RFE         0    0.0000  0.0000  -249.9172  -0.6190
2        GA       494  251.0353  0.6155     1.1180  -0.0034

Common Features (0): []


# GridSearch Find Best Params

In [8]:
from sklearn.model_selection import GridSearchCV

print("... GridSearch ...")

param_grid = {
    'n_estimators': [500, 1000, 2000],
    'learning_rate': [0.01, 0.03, 0.05],
    'num_leaves': [31, 50],
    'max_depth': [-1, 10, 20]
}

base_model = LGBMRegressor(n_jobs=1, verbose=-1)

valid_ga_feats = [f for f in list(selected_ga) if f in X_train.columns]

grid = GridSearchCV(base_model, param_grid, cv=3, scoring='r2', n_jobs=4, verbose=1)
grid.fit(X_train[valid_ga_feats], y_train)

print("\n--- Best Params ---")
print(f"Best Params: {grid.best_params_}")
print(f"Best R2 Score (Train CV): {grid.best_score_:.4f}")

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test[valid_ga_feats])
print(f"Test R2 Score: {r2_score(y_test, y_pred):.4f}")

... GridSearch ...
Fitting 3 folds for each of 54 candidates, totalling 162 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from lightgbm import LGBMRegressor

model_final = LGBMRegressor(
    learning_rate=0.01, 
    n_estimators=500, 
    num_leaves=50, 
    max_depth=-1, 
    random_state=2601, 
    n_jobs=1,
    verbose=-1
)

kf = KFold(n_splits=5, shuffle=True, random_state=2601)

valid_ga_feats = [f for f in list(selected_ga) if f in X_train.columns]
X_ga = X_clean[valid_ga_feats]

scores = cross_val_score(model_final, X_ga, y, cv=kf, scoring='r2', n_jobs=4)

print(f"Scores: {scores}")
print(f"Mean R2: {scores.mean():.4f} (+/- {scores.std():.4f})")

# Elbow GA Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from lightgbm import LGBMRegressor

best_params = {
    'learning_rate': 0.01,
    'n_estimators': 2000,
    'num_leaves': 50,
    'max_depth': -1,
    'random_state': 2601,
    'n_jobs': 1,
    'verbose': -1
}

valid_ga_feats = [f for f in list(selected_ga) if f in X_train.columns]

print("Đang xếp hạng features...")
ranker = LGBMRegressor(**best_params)
ranker.fit(X_train[valid_ga_feats], y_train)

imp_df = pd.DataFrame({
    'Feature': valid_ga_feats,
    'Importance': ranker.feature_importances_
}).sort_values(by='Importance', ascending=False)

sorted_feats = imp_df['Feature'].tolist()

steps = list(range(len(sorted_feats), 99, -50)) + list(range(90, 0, -10))
results = []

print(f"\nBắt đầu vòng lặp cắt giảm features ({len(steps)} vòng)...")

for k in steps:
    current_feats = sorted_feats[:k]
    
    model = LGBMRegressor(**best_params)
    model.fit(X_train[current_feats], y_train)
    
    y_pred = model.predict(X_test[current_feats])

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"   -> Dùng {k:3d} features: R2 = {r2:.4f} | RMSE = {rmse:.2f}")
    results.append({'Num_Features': k, 'R2': r2, 'RMSE': rmse})


df_results = pd.DataFrame(results).sort_values(by='Num_Features')
df_leaderboard = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)
csv_filename = 'result/feature_selection_results.csv'
df_results.to_csv(csv_filename, index=False)

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['R2'], marker='o', linewidth=2, color='blue')

best_row = df_results.loc[df_results['R2'].idxmax()]
plt.scatter(best_row['Num_Features'], best_row['R2'], color='red', s=150, zorder=5)
plt.annotate(f"Đỉnh: {best_row['R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['R2']), 
             xytext=(best_row['Num_Features']+20, best_row['R2']-0.01),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('Biểu đồ Elbow: Hiệu quả khi giảm dần số lượng Features', fontsize=14)
plt.xlabel('Số lượng Features', fontsize=12)
plt.ylabel('Độ chính xác (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_xaxis()
plt.show()

print("\nBẢNG XẾP HẠNG (SCORE GIẢM DẦN):")
print(df_leaderboard[['R2', 'Num_Features', 'RMSE']].head(10))

print("\nBẢNG THEO THỨ TỰ FEATURE (ÍT -> NHIỀU):")
print(df_results[['Num_Features', 'R2', 'RMSE']].head(10))

# Elbow RFECV Model

In [ ]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

print("\n--- START RFE WITH LEADERBOARD ---")
start = time.time()

model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=1,
    verbose=-1
)

rfecv = RFECV(
    estimator=model,
    step=20,
    cv=3,
    scoring='r2', 
    min_features_to_select=50,
    n_jobs=-1,
    verbose=1
)

rfecv.fit(X_train, y_train)

r2_scores = rfecv.cv_results_['mean_test_score']
n_scores = len(r2_scores)

feature_counts = [50 + i * 20 for i in range(n_scores)]

df_results = pd.DataFrame({
    'Num_Features': feature_counts,
    'Score_R2': r2_scores
})

df_leaderboard = df_results.sort_values(by='Score_R2', ascending=False).reset_index(drop=True)

selected_rfecv = X_train.columns[rfecv.support_]
print(f"\nTime Run: {time.time() - start:.2f} s")
print(f"Best Number of Features: {rfecv.n_features_}")
print(f"Best R2 Score: {df_leaderboard.iloc[0]['Score_R2']:.4f}")

print("\nLEADERBOARD (DESCENDING SCORE):")
print(df_leaderboard.head(10).to_string(index=False))

print("\nPROGRESS (BY FEATURE COUNT):")
print(df_results.head(10).to_string(index=False))

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['Score_R2'], marker='o', color='green', linewidth=2)

best_row = df_leaderboard.iloc[0]
plt.scatter(best_row['Num_Features'], best_row['Score_R2'], color='red', s=150, zorder=5)
plt.annotate(f"Best: {best_row['Score_R2']:.4f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['Score_R2']), 
             xytext=(best_row['Num_Features']+20, best_row['Score_R2']-0.005),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('RFECV Performance Curve', fontsize=14)
plt.xlabel('Number of Features Selected', fontsize=12)
plt.ylabel('Cross Validation Score (R2)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

print("\nSelected Features List:")
print(list(selected_rfecv))

df_results.to_csv('result/rfecv_results.csv', index=False)

# Compare before and after using partial correlation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor

ULTRA_HIGH_CORR = 0.999 

corr_matrix = X_train_clean.corr().abs()

ranker_temp = LGBMRegressor(n_estimators=100, verbose=-1, random_state=2601)
ranker_temp.fit(X_train_clean, y_train_clean)
importances = pd.Series(ranker_temp.feature_importances_, index=X_train_clean.columns)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = []

for column in upper.columns:
    correlated_cols = upper.index[upper[column] > ULTRA_HIGH_CORR].tolist()
    if correlated_cols:
        for other_col in correlated_cols:
            if other_col in to_drop: continue
            imp_col = importances.get(column, 0)
            imp_other = importances.get(other_col, 0)
            if imp_col < imp_other:
                to_drop.append(column)
                break 
            else:
                to_drop.append(other_col)

to_drop = list(set(to_drop))

print(f"Phat hien {len(to_drop)} features tuong quan > {ULTRA_HIGH_CORR}")
print(f"Vi du: {to_drop[:5]}")

voting_model = LGBMRegressor(
    learning_rate=0.01,
    n_estimators=2000,
    num_leaves=50,
    max_depth=-1,
    random_state=2601,
    n_jobs=1,
    verbose=-1
)

if len(to_drop) > 0:
    X_train_ultra = X_train_clean.drop(columns=to_drop)
    X_test_ultra = X_test.drop(columns=to_drop)

    print(f"Train thu tren {X_train_ultra.shape[1]} features...")
    voting_model.fit(X_train_ultra, y_train_clean)
    
    y_pred = voting_model.predict(X_test_ultra)
    new_r2 = r2_score(y_test, y_pred)
    
    old_r2 = 0.6313 
    
    print(f"Ket qua R2 moi: {new_r2:.4f}")
    
    methods = ['Original Clean', 'Filtered (>0.995)']
    scores = [old_r2, new_r2]
    colors = ['green', 'red']

    plt.figure(figsize=(8, 6))
    bars = plt.bar(methods, scores, color=colors, width=0.5)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.4f}',
                 ha='center', va='bottom', fontsize=12, fontweight='bold')

    plt.title('Hieu qua giua giu nguyen va loc tuong quan', fontsize=14)
    plt.ylabel('R2 Score', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(0, 0.8)
    plt.show()

else:
    print("Khong co features nao qua giong nhau de xoa.")

---

# EXTRA MODEL 
# MAE (Mean Absolute Error)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.feature_selection import RFE
import time

best_params = {
    'learning_rate': 0.01,
    'n_estimators': 2000,
    'num_leaves': 50,
    'max_depth': -1,
    'random_state': 2601,
    'n_jobs': 1,
    'verbose': -1
}

fast_params = {
    'learning_rate': 0.1,
    'n_estimators': 100,
    'num_leaves': 31,
    'random_state': 2601,
    'n_jobs': -1,
    'verbose': -1
}

print("Ranking features (Fast Mode)...")
start_time = time.time()

rfe_selector = RFE(estimator=LGBMRegressor(**fast_params), 
                   n_features_to_select=1, 
                   step=0.1, 
                   verbose=0)

rfe_selector.fit(X_train, y_train) 

rfe_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Rank': rfe_selector.ranking_
}).sort_values(by='Rank', ascending=True)

sorted_feats = rfe_df['Feature'].tolist()
print(f"Ranking done in {time.time() - start_time:.1f}s")

steps = list(range(len(sorted_feats), 99, -50)) + list(range(90, 0, -10))
results = []

print(f"\nStarting MAE Loop ({len(steps)} steps)...")

for k in steps:
    current_feats = sorted_feats[:k]
    
    model = LGBMRegressor(**best_params)
    model.fit(X_train[current_feats], y_train)
    
    y_pred = model.predict(X_test[current_feats])
    mae = mean_absolute_error(y_test, y_pred)

    print(f"   -> Top {k:3d} features: MAE = {mae:.2f}")
    results.append({'Num_Features': k, 'MAE': mae})

df_results = pd.DataFrame(results).sort_values(by='Num_Features')
df_leaderboard = df_results.sort_values(by='MAE', ascending=True).reset_index(drop=True)

csv_filename = 'result/rfe_mae_results.csv'
df_results.to_csv(csv_filename, index=False)

plt.figure(figsize=(12, 6))
plt.plot(df_results['Num_Features'], df_results['MAE'], marker='o', linewidth=2, color='purple')

best_row = df_results.loc[df_results['MAE'].idxmin()]
plt.scatter(best_row['Num_Features'], best_row['MAE'], color='red', s=150, zorder=5)
plt.annotate(f"Best MAE: {best_row['MAE']:.2f}\n({int(best_row['Num_Features'])} feats)", 
             (best_row['Num_Features'], best_row['MAE']), 
             xytext=(best_row['Num_Features']+20, best_row['MAE']+5),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.title('RFE Feature Selection (MAE Metric)', fontsize=14)
plt.xlabel('Number of Features', fontsize=12)
plt.ylabel('Mean Absolute Error (MAE)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.gca().invert_xaxis()
plt.show()

print("\nTOP 5 CONFIGURATIONS (LOWEST MAE):")
print(df_leaderboard.head(5))

print("\nLOOP PROGRESS:")
print(df_results.head(10))